In [37]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import io
import datetime
import statsmodels.formula.api as smf
import datetime

In [41]:
df = pd.read_csv('final_project_data.csv', parse_dates = True, index_col = 0)
df.tail(15)

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2023-12-12,4.732,4.506,NaN,3.1,NaN,NaN
2023-12-13,4.429,4.365,NaN,NaN,NaN,NaN
2023-12-14,4.389,4.327,NaN,NaN,NaN,NaN
2023-12-15,4.446,4.267,NaN,NaN,NaN,NaN
2023-12-18,4.447,4.296,NaN,NaN,102.4301,93.9
2023-12-19,4.441,4.265,NaN,NaN,NaN,NaN
2023-12-20,4.333,4.089,3.9,NaN,NaN,NaN
2023-12-21,4.348,4.044,NaN,NaN,NaN,NaN
2023-12-22,4.324,4.030,NaN,NaN,NaN,NaN


In [42]:
#we find the first value of CPI in order the delet the NaN values that are before that date
initial_date = df['UK YoY CPI'].first_valid_index() 
#.fist_valid_index() returns the first non-NaN value
initial_date

Timestamp('2003-06-17 00:00:00')

In [43]:
#im gonna transform initial_date to first day of the last month 
initial_date = initial_date.replace(day=1)
#We reduce 1 month to this date
initial_date = initial_date - pd.DateOffset(months=1)
initial_date

Timestamp('2003-05-01 00:00:00')

In [44]:
#we are gonna drop all the values before initial_date
df = df[initial_date:]
df.head()
252

252

In [45]:
#Im gonna try transform USD2YB and UK2YB to a daily frequency
df['USD2YB'] = ((df['USD2YB']/100)+1)**(1/126)
df['UK2YB'] = ((df['UK2YB']/100)+1)**(1/126)

In [46]:
#Im gonna transform the USD2YB and UK2YB to a log return
df['USD2YB'] = np.log(df['USD2YB'])
df['UK2YB'] = np.log(df['UK2YB'])


In [47]:
df

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-01,0.000116,0.000279,NaN,NaN,NaN,NaN
2003-05-02,0.000123,0.000283,NaN,NaN,NaN,NaN
2003-05-05,0.000121,NaN,NaN,NaN,NaN,NaN
2003-05-06,0.000113,0.000284,NaN,NaN,NaN,NaN
2003-05-07,0.000111,0.000280,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2023-12-27,0.000330,0.000308,NaN,NaN,NaN,NaN
2023-12-28,0.000332,0.000310,NaN,NaN,NaN,NaN
2023-12-29,0.000330,0.000308,NaN,NaN,NaN,NaN


In [48]:
nan_values = df.isnull().sum()
print("NaN values in each column:")
print(nan_values)



NaN values in each column:
USD2YB                  299
UK2YB                   246
UK YoY CPI             5221
US YoY CPI             5220
US Industrial Index    5219
UK industrial Index    5220
dtype: int64


In [49]:
#Im gonna resample the data to a monthly frequency. For USD2YB and UK2YB we are gonna use the sum of the log returns, and for the rest of the columns we are gonna use the last value of the month.

df2 = df.resample('M').agg({'USD2YB':'sum', 'UK2YB':'sum', 'UK YoY CPI':'last', 'US YoY CPI':'last', 'UK industrial Index':'last', 'US Industrial Index':'last'})
df2

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,UK industrial Index,US Industrial Index
Date,,,,,,
2003-05-31,0.002300,0.005518,NaN,2.2,101.1,90.4281
2003-06-30,0.002001,0.005601,1.2,2.1,99.8,90.4073
2003-07-31,0.002527,0.006387,1.1,2.1,100.4,90.5218
2003-08-31,0.003007,0.006292,1.3,2.1,101.0,90.9896
2003-09-30,0.002777,0.007110,1.4,2.2,101.2,90.7858
...,...,...,...,...,...,...
2023-09-30,0.007813,0.008113,6.7,3.7,95.0,103.2411
2023-10-31,0.008264,0.008272,6.7,3.7,94.9,103.3346
2023-11-30,0.007962,0.007883,4.6,3.2,93.7,102.4592


In [50]:
# Specify the columns to be shifted
columns_to_shift = ['UK YoY CPI', 'US YoY CPI', 'US Industrial Index', 'UK industrial Index']

# Shift the specified columns using .shift(-1)
df2[columns_to_shift] = df2[columns_to_shift].shift(-1)

# Display the resulting DataFrame with shifted columns
df2

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,UK industrial Index,US Industrial Index
Date,,,,,,
2003-05-31,0.002300,0.005518,1.2,2.1,99.8,90.4073
2003-06-30,0.002001,0.005601,1.1,2.1,100.4,90.5218
2003-07-31,0.002527,0.006387,1.3,2.1,101.0,90.9896
2003-08-31,0.003007,0.006292,1.4,2.2,101.2,90.7858
2003-09-30,0.002777,0.007110,1.4,2.3,101.4,91.3829
...,...,...,...,...,...,...
2023-09-30,0.007813,0.008113,6.7,3.7,94.9,103.3346
2023-10-31,0.008264,0.008272,4.6,3.2,93.7,102.4592
2023-11-30,0.007962,0.007883,3.9,3.1,93.9,102.4301


In [51]:
#we get rid of the last two rows of the dataframes
df2 = df2[:-2]

In [53]:
df2

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,UK industrial Index,US Industrial Index
Date,,,,,,
2003-05-31,0.002300,0.005518,1.2,2.1,99.8,90.4073
2003-06-30,0.002001,0.005601,1.1,2.1,100.4,90.5218
2003-07-31,0.002527,0.006387,1.3,2.1,101.0,90.9896
2003-08-31,0.003007,0.006292,1.4,2.2,101.2,90.7858
2003-09-30,0.002777,0.007110,1.4,2.3,101.4,91.3829
...,...,...,...,...,...,...
2023-07-31,0.007519,0.008350,6.8,3.2,95.4,103.1900
2023-08-31,0.008762,0.008603,6.7,3.7,95.0,103.2411
2023-09-30,0.007813,0.008113,6.7,3.7,94.9,103.3346


In [52]:
df2.isnull().sum()

USD2YB                 0
UK2YB                  0
UK YoY CPI             0
US YoY CPI             0
UK industrial Index    0
US Industrial Index    0
dtype: int64

# using lambda x 

In [130]:
# Convert the index to datetime
df.index = pd.to_datetime(df.index)

# Resample the data to monthly frequency 
df = df.resample('M').apply(lambda x: (x + 1).prod() - 1)

# Display the resulting monthly data
df

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-31,-0.616739,1.538217,1.400000,2.114286,91.455700,102.514286
2003-06-30,-0.792333,1.417429,1.336364,1.872727,91.841955,102.436364
2003-07-31,-0.548522,1.560870,1.300000,1.873913,92.156304,101.800000
2003-08-31,-0.172773,2.053364,1.361905,1.900000,92.260657,102.542857
2003-09-30,-0.306364,2.156227,1.334783,1.778261,92.643691,102.930435
...,...,...,...,...,...,...
2023-09-30,3.038045,2.983545,4.250000,3.120000,102.447130,93.820000
2023-10-31,3.083091,2.851818,4.250000,3.120000,102.447130,93.820000
2023-11-30,2.893478,2.614522,4.250000,3.120000,102.447130,93.820000


In [131]:
df.isnull().sum()

USD2YB                 0
UK2YB                  0
UK YoY CPI             0
US YoY CPI             0
US Industrial Index    0
UK industrial Index    0
dtype: int64

In [132]:
df.isnull().sum()

USD2YB                 0
UK2YB                  0
UK YoY CPI             0
US YoY CPI             0
US Industrial Index    0
UK industrial Index    0
dtype: int64

# using geometric mean

In [133]:
# Assuming df is your DataFrame with a datetime index
# and columns representing different financial indicators, including interest rates

# Convert the index to datetime if needed
df.index = pd.to_datetime(df.index)

# Define a custom aggregation function for interest rates using the geometric mean
def geometric_mean(arr):
    return np.exp(np.mean(np.log(1 + arr))) - 1

# Resample to monthly frequency and apply the aggregation function
df= df.resample('M').agg({
    'USD2YB': 'mean',               # Example non-interest rate column
    'UK2YB': 'mean',        # Example interest rate column
    'UK YoY CPI': 'mean',           # Add other non-interest rate columns
    'US YoY CPI': 'mean',
    'US Industrial Index': 'mean',
    'UK industrial Index': 'mean'
})

In [134]:
df

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-31,-0.616739,1.538217,1.400000,2.114286,91.455700,102.514286
2003-06-30,-0.792333,1.417429,1.336364,1.872727,91.841955,102.436364
2003-07-31,-0.548522,1.560870,1.300000,1.873913,92.156304,101.800000
2003-08-31,-0.172773,2.053364,1.361905,1.900000,92.260657,102.542857
2003-09-30,-0.306364,2.156227,1.334783,1.778261,92.643691,102.930435
...,...,...,...,...,...,...
2023-09-30,3.038045,2.983545,4.250000,3.120000,102.447130,93.820000
2023-10-31,3.083091,2.851818,4.250000,3.120000,102.447130,93.820000
2023-11-30,2.893478,2.614522,4.250000,3.120000,102.447130,93.820000
